This notebook uses demo data to explain a concept that you will come across often in the type3 rules. It will guide you through rules that sound like "within a CINdetails group, no startDate should be between the start and end dates of any other group. if the end date is missing, replace the end date with reference date"

In [1]:
import pandas as pd

Let us consider sample data and shown below, and seek to implement this sample rule:

## In every group, flag column B values that exist within the intervals of any other colA-colB combinations in that group

In [2]:
df = pd.DataFrame([
    {"id":"id1", "a": 1, "b": 2}, #0
    {"id":"id1", "a": 3, "b": 4}, #1
    {"id":"id1", "a": 13, "b": 9}, #2
    
    {"id":"id2", "a": 5, "b": 7}, #3 fail . exists between 6 and 8.
    {"id":"id2", "a": 6, "b": 8}, #4
])

print(df)

    id   a  b
0  id1   1  2
1  id1   3  4
2  id1  13  9
3  id2   5  7
4  id2   6  8


Preserving the initial index positions enables us to ensure that a row is not compared to itself. This is because, each value will exists in its own interval and hence wrongly flag the error.

In [3]:
df.index.name = "ROW_ID"
df.reset_index(inplace=True)
print(df)

   ROW_ID   id   a  b
0       0  id1   1  2
1       1  id1   3  4
2       2  id1  13  9
3       3  id2   5  7
4       4  id2   6  8


To get all the possible combinations of the values with themselves, we merge the dataframe with itself on the columns that define what a "group" means in our case. 
If the group needed in your rule is a subgroup e.g CPP, then you'll have to merge on all the parent groups too. That is, LAchildID, CINdetailsID and CPPID will have to be included to define a CPP group.

In [4]:
df_check = df.reset_index()[["ROW_ID","id","b"]].merge(df.reset_index()[["ROW_ID","id","a", "b"]], on=["id"], suffixes=["", "_check"])

# see what your data looks like now
print(df_check)

    ROW_ID   id  b  ROW_ID_check   a  b_check
0        0  id1  2             0   1        2
1        0  id1  2             1   3        4
2        0  id1  2             2  13        9
3        1  id1  4             0   1        2
4        1  id1  4             1   3        4
5        1  id1  4             2  13        9
6        2  id1  9             0   1        2
7        2  id1  9             1   3        4
8        2  id1  9             2  13        9
9        3  id2  7             3   5        7
10       3  id2  7             4   6        8
11       4  id2  8             3   5        7
12       4  id2  8             4   6        8


The merge result shows that suffixes are only applied to the common columns between the merge tables, so column a is unaffected. We have applied an empty string suffix (nothing changes) to the left hand side (b) and a "_check" suffix to the right hand side(b_check).

We prevent positions from being compared to each other by removing the rows where the initial index values (ROW_IDs), of the the merged tables, are the same.

In [5]:
df_check = df_check[~(df_check["ROW_ID"]==df_check["ROW_ID_check"])]
print(df_check)

    ROW_ID   id  b  ROW_ID_check   a  b_check
1        0  id1  2             1   3        4
2        0  id1  2             2  13        9
3        1  id1  4             0   1        2
5        1  id1  4             2  13        9
6        2  id1  9             0   1        2
7        2  id1  9             1   3        4
10       3  id2  7             4   6        8
11       4  id2  8             3   5        7


The idea is to filter out df_check until all it has are the failing positions. 
Since we have all the possible combinations for column a and column b for every column b value in a group, we can now check if column b's value exists in the columna-columnb interval of the other combinations in its group.

In [6]:
df_check = df_check[(df_check["b"]>=df_check["a"]) & (df_check["b"]<=df_check["b_check"]) ]
print(df_check)

    ROW_ID   id  b  ROW_ID_check  a  b_check
10       3  id2  7             4  6        8


We can then proceed to create ERROR_IDs for the failing locations and then map these to the original dataframe. Your ERROR_ID should contain all the columns that define a group ( from parent group to subgroup, in that order) and the base column being compared (column b in this case).

In [7]:
error_ids = tuple(
    zip(
        df_check["id"], df_check["b"]
    )
)
print(error_ids)

(('id2', 7),)


We use the exact combination of columns in error_ids to create an ERROR_ID column in our original dataframe, df.

In [8]:
# back to original dataframe
df["ERROR_ID"] = tuple(
    zip(
        df["id"], df["b"]
    )
)
print(df)

   ROW_ID   id   a  b  ERROR_ID
0       0  id1   1  2  (id1, 2)
1       1  id1   3  4  (id1, 4)
2       2  id1  13  9  (id1, 9)
3       3  id2   5  7  (id2, 7)
4       4  id2   6  8  (id2, 8)


We select out all the rows of the original dataframe whose ERROR_IDs exist in the sequence of failing positions (error_ids)

In [9]:
df_issues = df[df["ERROR_ID"].isin(error_ids)]
print(df_issues)

   ROW_ID   id  a  b  ERROR_ID
3       3  id2  5  7  (id2, 7)


Lastly, all the index positions that have the same failing ID are grouped together so that they can be pushed.

In [10]:
df_issues.groupby(["ERROR_ID"])["ROW_ID"].apply(list)

ERROR_ID
(id2, 7)    [3]
Name: ROW_ID, dtype: object

## Alternatively, 

In [11]:
# alternatively, you could proceed from df_check directly to df_issues.

df_check["ERROR_ID"] = tuple(
    zip(
        df_check["id"], df_check["b"]
    )
 )
print(df_check)

    ROW_ID   id  b  ROW_ID_check  a  b_check  ERROR_ID
10       3  id2  7             4  6        8  (id2, 7)


This method now allows us to link up the rows that failed (i.e column b, row 3) to the locations that caused the failure (column a and column b, row 4).

Side note: the values 3 and 4 come from the values of ROW_ID and ROW_ID check in the failing data (df_check), respectively.

In [12]:
df_issues = df_check.groupby(["ERROR_ID"])["ROW_ID"].apply(list)
print(df_issues)
# push_type3(table=TableName, columns=["a"], row_df=df_issues)

ERROR_ID
(id2, 7)    [3]
Name: ROW_ID, dtype: object


In [13]:
df_issues_check = df_check.groupby(["ERROR_ID"])["ROW_ID_check"].apply(list)
print(df_issues_check)
# push_type3(table=TableName, columns["a", "b"], row_df = df_issues_check)

ERROR_ID
(id2, 7)    [4]
Name: ROW_ID_check, dtype: object
